# Data Augmentation for Toxic Spans Detection
### Author: Yakoob Khan



### Load Code from Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "drive/My Drive/system/src"

/content/drive/My Drive/system/src


### Install dependencies

In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 890kB 11.5MB/s 
     |████████████████████████████████| 3.2MB 44.8MB/s 
     |████████████████████████████████| 1.9MB 48.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=0032d8e031e7d7265177fc3ed92869b947aab76c2d471750610a09bbf5f8dcc3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


### GPU Info

In [4]:
# Credit: https://colab.research.google.com/notebooks/pro.ipynb
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Sun Feb 28 20:29:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Create Augmented Dataset using Easy Data Augmentation 

In [5]:
!python3 './eda.py' \
 --synonym_replacement 'True' \
 --random_insertion 'False' \
 --random_deletion 'False' \
 --random_swap 'False' \
 --alpha 0.8 \
 --train_dir '../data/tsd_train.csv' \
 --output './augmented_data/tsd_eda_sr.csv'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.

> Loading 7939 examples located at '../data/tsd_train.csv'

> Written 15878 posts to ./augmented_data/tsd_eda_sr.csv file!


### Fine-tune BERT using augmented EDA dataset

In [6]:
!python3 './train_bert.py' \
  --model_type 'bert-base-cased' \
  --train_dir './augmented_data/tsd_eda_sr.csv' \
  --dev_dir '../data/tsd_trial.csv' \
  --test_dir '../data/tsd_test.csv' \
  --epochs 1.92 \
  --warm_up_steps 500 \
  --learning_rate 5e-5 \
  --weight_decay 0.01 \
  --batch_size 16 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2021-02-28 20:29:47.043205: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Downloading: 100% 213k/213k [00:00<00:00, 636kB/s]
Downloading: 100% 436k/436k [00:00<00:00, 1.27MB/s]
Downloading: 100% 433/433 [00:00<00:00, 379kB/s]
Downloading: 100% 436M/436M [00:11<00:00, 36.9MB/s]
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with

### Create augmented dataset with HateXplain's dataset

In [7]:
!python3 './augment_with_hateXplain_dataset.py' \
  --hateXplain '../data/hateXplain.json' \
  --train_dir '../data/tsd_train.csv' \
  --output './augmented_data/tsd_hateXplain.csv' 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.

> Loading 7939 examples located at '../data/tsd_train.csv'

> Written 19354 posts to ./augmented_data/tsd_hateXplain.csv file!


### Fine-tune BERT with augmented HateXplain's dataset

In [8]:
!python3 './train_bert.py' \
  --model_type 'bert-base-cased' \
  --train_dir './augmented_data/tsd_hateXplain.csv' \
  --dev_dir '../data/tsd_trial.csv' \
  --test_dir '../data/tsd_test.csv' \
  --epochs 1.92 \
  --warm_up_steps 500 \
  --learning_rate 5e-5 \
  --weight_decay 0.01 \
  --batch_size 16 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2021-02-28 21:08:33.043178: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoi